### Finding curvature distance from all airports in the US to a selected airport excluding those located within the same state.

In [1]:
# Import necessary libraries
import concurrent.futures
import geopandas as gpd
import json
import math

In [2]:
# Constants
EARTH_RADIUS = 6371.0
NUM_WORKERS = 14

In [3]:
# Load shapefile
shapefile_path = "kx-us-airports-SHP/us-airports.shp"
gdf = gpd.read_file(shapefile_path)

In [4]:
# Extract columns
print("Columns:", gdf.columns)

gdf = gdf.rename(columns={
    'ST_POSTAL': 'state',
    'FULLNAME': 'name',
    'LATITUDE': 'latitude',
    'LONGITUDE': 'longitude'
})
gdf = gdf.dropna(subset=['latitude', 'longitude', 'state'])

Columns: Index(['LOCID', 'SITE_NO', 'FULLNAME', 'FAA_ST', 'LAN_FA_TY', 'EFF_DATE',
       'FAA_REGION', 'FAA_DISTRI', 'STATE_NAME', 'ST_POSTAL', 'STFIPS',
       'COUNTY_NAM', 'COUNTY_ST', 'CITY_NAME', 'OWNER_TYPE', 'FAC_USE',
       'FAC_CYSTZP', 'ELEV', 'AERO_CHART', 'CBD_DIST', 'CBD_DIR', 'ACT_DATE',
       'CERT_TYPE', 'FED_AGREE', 'INTERNATIO', 'CUST_LNDG', 'JOINT_USE',
       'MIL_LNDG_R', 'MIL_INT', 'CNTL_TWR', 'S_ENG_GA', 'M_ENG_GA',
       'JET_EN_GA', 'HELICOPTER', 'OPER_GLIDE', 'OPER_MIL', 'ULTRALIGHT',
       'COMM_SERV', 'AIR_TAXI', 'LOCAL_OPS', 'ITIN_OPS', 'MIL_OPS', 'LATITUDE',
       'LONGITUDE', 'VERSION', 'CY_06_ENP', 'geometry'],
      dtype='object')


In [5]:
# Compute curvature distance using Haversine formula 
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = (math.sin(dlat / 2) ** 2 +
         math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return EARTH_RADIUS * c


In [8]:
# Compute total distance
def compute_total_distance(airport1, airports):
    name1 = airport1['name']
    state1 = airport1['state']
    lat1 = airport1['latitude']
    lon1 = airport1['longitude']
    accumulated_distance = 0.0

    for _, airport2 in airports.iterrows():
        if airport1.name == airport2.name:
            continue
        if airport2['state'] == state1:
            continue

        lat2 = airport2['latitude']
        lon2 = airport2['longitude']
        name2 = airport2['name']
        dist = haversine(lat1, lon1, lat2, lon2)
        accumulated_distance += dist

        # print(f"Distance {name1} -> {name2}: {dist:.2f} km | Accumulated: {accumulated_distance:.2f} km")

    print(f"Total distance for {name1}: {accumulated_distance:.2f} km")
    
    return (name1, accumulated_distance)

In [ ]:
# Compute in parallel
total_distance_per_airport = {}
num_workers = min(NUM_WORKERS, len(gdf)) 

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
    futures = [executor.submit(compute_total_distance, row, gdf) for _, row in gdf.iterrows()]

    for future in concurrent.futures.as_completed(futures):
        name, total_dist = future.result()
        total_distance_per_airport[name] = total_dist

In [ ]:
# Final dictionary output
print("\n=== Total distance per airport (km) ===")

for k, v in total_distance_per_airport.items():
    print(f"{k}: {v:.2f}")

In [ ]:
# Save result to a JSON file
with open("total_distance_per_airport.json", "w") as f:
    json.dump(total_distance_per_airport, f, indent=2)

Index(['LOCID', 'SITE_NO', 'FULLNAME', 'FAA_ST', 'LAN_FA_TY', 'EFF_DATE',
       'FAA_REGION', 'FAA_DISTRI', 'STATE_NAME', 'ST_POSTAL', 'STFIPS',
       'COUNTY_NAM', 'COUNTY_ST', 'CITY_NAME', 'OWNER_TYPE', 'FAC_USE',
       'FAC_CYSTZP', 'ELEV', 'AERO_CHART', 'CBD_DIST', 'CBD_DIR', 'ACT_DATE',
       'CERT_TYPE', 'FED_AGREE', 'INTERNATIO', 'CUST_LNDG', 'JOINT_USE',
       'MIL_LNDG_R', 'MIL_INT', 'CNTL_TWR', 'S_ENG_GA', 'M_ENG_GA',
       'JET_EN_GA', 'HELICOPTER', 'OPER_GLIDE', 'OPER_MIL', 'ULTRALIGHT',
       'COMM_SERV', 'AIR_TAXI', 'LOCAL_OPS', 'ITIN_OPS', 'MIL_OPS', 'LATITUDE',
       'LONGITUDE', 'VERSION', 'CY_06_ENP', 'geometry'],
      dtype='object')
